# ДЗ №3
Задачи линейной алгебры. Решение СЛАУ

In [6]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
from typing import Tuple

In [7]:
def cond(m: np.array) -> float:
    M_inv = np.linalg.inv(m)
    return norm(M_inv) * norm(m)

In [8]:
def generate_rotation_matrix(i: int, j: int, z: np.ndarray) -> np.ndarray:
    n = z.shape[0]
    sq = np.sqrt(z[i]**2 + z[j]**2)
    cos_phi = z[i] / sq
    sin_phi = - z[j] / sq

    T = np.identity(n)
    T[i, i] = cos_phi
    T[j, j] = cos_phi
    T[j, i] = sin_phi
    T[i, j] = -sin_phi

    return T

def generate_Q(A: np.ndarray) -> np.ndarray:
    n = A.shape[0]
    A_copy = A.copy()
    M = np.eye(n)
    for i in range(n):
        for j in range(i + 1, n):
            T = generate_rotation_matrix(i, j, A_copy[:, i])
            M = np.matmul(M, T.T)
            A_copy = np.matmul(T, A_copy)
    return M

def qr_decomposition(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    Q = generate_Q(A)
    return Q, np.matmul(Q.T, A)

In [9]:
def solve_for_upper_triangle(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    n = A.shape[0]
    A = np.c_[A, b]
    solutions = np.array([A[n - 1][n] / A[n - 1][n - 1]])
    for i in range(n - 2, -1, -1):
        x = A[i][n] - np.dot(solutions, A[i, (i+1):n])
        solutions = np.insert(solutions, 0, x / A[i, i])
    solutions = solutions.T
    return solutions

def solve_with_qr(A: np.ndarray, u: np.ndarray) -> Tuple[np.ndarray, pd.DataFrame]:
    n = A.shape[0]
    Q, R = qr_decomposition(A)
    u_modified = np.matmul(Q.T, u)
    x = solve_for_upper_triangle(R, u_modified)

    stat = pd.DataFrame({
        'Relative error (%)': [norm(np.matmul(A, x).reshape((n, 1)) - u) / norm(u)],
        'Cond_A': [cond(A)],
        'Cond_Q': [cond(Q)],
        'Cond_R': [cond(R)]
    })

    return x, stat

In [10]:
A = np.array([
    [1,2,3],
    [3,4,5],
    [9,7,1]
])

u = np.array([
    [1],
    [2],
    [9]
])

x, stat = solve_with_qr(A, u)

In [11]:
stat

,Relative error (%),Cond_A,Cond_Q,Cond_R
0,2.141588e-16,112.203777,3.0,112.203777


# Тестирование

In [15]:
from scipy.linalg import hilbert

In [12]:
# Пример из презентации
A = np.array([
    [1, 0.99],
    [0.99, 0.98]
])

b = np.array([
    [1.99],
    [1.97]
])

In [13]:
_, stat = solve_with_qr(A, b)
stat

,Relative error (%),Cond_A,Cond_Q,Cond_R
0,1.121423e-16,39206.0,2.0,39206.0


In [16]:
# Матрица Гильберта
sizes = [3, 5, 10, 20, 100]
H_n = [hilbert(n) for n in sizes]
stats = []
for H, n in zip(H_n, sizes):
    u = np.matmul(H, np.ones(n))
    _, stat = solve_with_qr(H, u)
    stats.append(stat)
s = pd.concat(stats)
s.index = sizes
s


,Relative error (%),Cond_A,Cond_Q,Cond_R
3,0.825696,5.261588e+02,3.0,5.261588e+02
5,1.239001,4.808491e+05,5.0,4.808491e+05
10,1.967242,1.633364e+13,10.0,1.633241e+13
20,2.967396,2.364931e+18,20.0,1.786334e+18
100,7.071205,7.604489e+18,100.0,4.650058e+19
